# Experiment 2 of the NeuroScienceJargon Paper
This paper shows the experiment conducted to kind of replicate the results of the paper: Deconstructing the seductive allure of neuroscience explanations. 
In this experiment, subjects are always given one good and one bad explanation for a phenomenon. There are three conditions: 
* Both explanations containing neuroscience jargon
* No explanation containing neuroscience jargon
* Only the bad explanation containing neuroscience jargon

subjects are asked to pick the explanation they find more satisfying, and are asked to explain in one or two sentences why they picked that explanation.

In [1]:
from utils_openAI import *
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) #for nicer output

Get the Stimulus and the explanations ready first. 

In [2]:
stimulus = ['Babies were seated on their mothers’ laps in front of a stage. Researchers used a camera to track where the babies were looking. The babies saw a hand reach out and place one doll on the stage. Then a screen was raised, hiding the doll. A hand reached out again and placed a second doll on the stage, out of sight behind the screen. Then the screen dropped. In some cases, there were two dolls on the stage, as there should be, and in some cases there was only one doll. The researchers found that the babies looked much longer at the stage when there was only one doll than when there were two dolls. This looking-time difference between one doll and two dolls lead the researchers to conclude that babies can calculate 1 + 1 = 2.', 
'Subjects sat at a computer screen. They saw a rapidly flashing series of pictures of faces. Somewhere in this series of faces there were two pictures of houses. Subjects had to press a button each time they saw a house. When the two houses were far apart in the sequence, the subjects were very good at this task. But when the houses were presented close together in the sequence, subjects failed to press the button for the second house. The researchers call this phenomenon “attentional blink.”', 
'Researchers recruited equal numbers of male and female participants. The participants took a series of spatial  reasoning tasks and were interviewed. The researchers determined that men are better at spatial reasoning in general. From the interviews, they discovered that the men had played more sports in their childhood on average than the women.']


In [3]:
#explanation
#Without Neuroscience Short Good
without_good = ["The researchers claim this happens because the babies had formed an expectation about how many dolls there should be on the stage. The babies knew there should be two dolls, and their surprise at seeing only one led to their looking longer.",
      "The researchers claim that this phenomenon occurs because the subjects were still processing the first house and missed seeing the second house because they did not have enough attentional resources left.", 
      "The researchers conclude that the difference in involvement in sports explains the gender difference in spatial reasoning abilities."]

#Without Neuroscience Short Bad
without_bad = ["The researchers claim this happens because the amount of time the babies spent looking at the stage is directly proportional to how much they liked the display. The researchers used this timing data to calculate babies’ preference for the single doll.", 
      "The researchers claim that this phenomenon occurs because the second house appeared later in the sequence than the first house, and this temporal relationship between the two houses caused the attentional blink.", 
      "The researchers conclude that women’s poor performance relative to men’s explains the gender difference in spatial reasoning abilities."]


#With Neuroscience Short Good
with_good = ["Scans of the babies’ brains show that the parietal lobe, known to be involved in math, governed the babies’ expectations about how many dolls there should be. They expected two, so they were surprised to see one, so they looked longer.", 
      "Researchers concluded that this occurs because of frontal lobe areas, previously shown to mediate attention. Subjects were still processing the first house and missed the second because they had insufficient attentional resources.", 
      "Brain scans of the right premotor area, known to be involved in spatial tasks, indicate that the difference in sports involvement explains this gender difference."]


#With Neuroscience Short Bad
with_bad = ["Scans of the babies’ brains show that the parietal lobe, known to be involved in math, governed how long babies looked at the stage. Researchers used this timing data, which is proportional to babies’ liking of the display, to calculate their preferences.", 
      "Researchers concluded that this occurs because of frontal lobe areas, previously shown to mediate attention. The second house appeared later in the sequence. This temporal relationship between the two houses caused the attentional blink.", 
      "Brain scans of the right premotor area, known to be involved in spatial tasks, indicate that women’s poor performance relative to men’s explains this gender difference."]



#make into a dict of lists
explanations_s2 = {"Without Neuroscience Short Good":without_good, "Without Neuroscience Short Bad":without_bad,  "With Neuroscience Short Good":with_good, "With Neuroscience Short Bad":with_bad}


Test whether the functions work to create the prompts and conditions

In [21]:
l, c, n, g = make_condition_exp2(stimulus, explanations_s2)
print(l, c, n, g)

['Subjects sat at a computer screen. They saw a rapidly flashing series of pictures of faces. Somewhere in this series of faces there were two pictures of houses. Subjects had to press a button each time they saw a house. When the two houses were far apart in the sequence, the subjects were very good at this task. But when the houses were presented close together in the sequence, subjects failed to press the button for the second house. The researchers call this phenomenon “attentional blink.”', 'The researchers claim that this phenomenon occurs because the subjects were still processing the first house and missed seeing the second house because they did not have enough attentional resources left.', 'Researchers concluded that this occurs because of frontal lobe areas, previously shown to mediate attention. The second house appeared later in the sequence. This temporal relationship between the two houses caused the attentional blink.'] Mixed 1 True


In [22]:
print(create_prompt_exp2(l, g))

#####Phenomenon#####:
Subjects sat at a computer screen. They saw a rapidly flashing series of pictures of faces. Somewhere in this series of faces there were two pictures of houses. Subjects had to press a button each time they saw a house. When the two houses were far apart in the sequence, the subjects were very good at this task. But when the houses were presented close together in the sequence, subjects failed to press the button for the second house. The researchers call this phenomenon “attentional blink.”
#####Explanation 1#####:
The researchers claim that this phenomenon occurs because the subjects were still processing the first house and missed seeing the second house because they did not have enough attentional resources left.
#####Explanation 2#####:
Researchers concluded that this occurs because of frontal lobe areas, previously shown to mediate attention. The second house appeared later in the sequence. This temporal relationship between the two houses caused the attenti

Create the instructions for GPT

In [6]:
instruction = '''You are a research subject participating in an experiment where you will be given a phenomenon and 2 explanations for the phenomenon. 
The prompt will show you the text "#####Phenomenon#####:" before the phenomenon,  "#####Explanation 1#####:" before the first explanation, and 
#####Explanation 2#####: before the second explanation. Your task is to pick the explanation that you find more satisfying. To do that, please answer with
[1] if you prefer explanation number 1, and [2] if you prefer explanation number 2, and [0] if you find the two explanations equally satisfying. 
After you have made your choice, please also explain in one or two sentences why you made that choice. 
Return your answer in the following format: [int]-Explanation why you chose that answer.'''

Run the Experiment.

In [24]:
#create empty pandas with columns stimulus_number, condition, good_first, answer 
df = pd.DataFrame(columns = ['stimulus_number', 'condition', 'good_first', 'answer'])
for i in range(400):
    #create condition
    cond_list, condition, num_stimulus, good_first = make_condition_exp2(stimulus, explanations_s2)
    #create prompt
    prompt = create_prompt_exp2(cond_list, good_first)

    #set OPEN ai API key
    openai.api_key = get_api_key(personal = False)
    #get answer from GPT
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": instruction},
            {"role": "user", "content": prompt}
        ], 
        timeout = 60, #set timeout to 60 seconds
        temperature = 0.2 #set temperature to 0.2 to have little randomless only
    )
    #get response
    answer = response['choices'][0]['message']['content']

    #create a new pd df with 1 row as the one outside the loop with values filled in
    df_temp = pd.DataFrame([[num_stimulus, condition, good_first, answer]], columns = ['stimulus_number', 'condition', 'good_first', 'answer'])
    #append to df
    df = pd.concat([df, df_temp])
    #save csv file after every itteration
    #df.to_csv("NeuroExp2_1.csv") #stopped working after 153
    #df.to_csv("NeuroExp2_2.csv") #stopped working after 115
    #df.to_csv("NeuroExp2_3.csv")
    

CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS API key
CSS 

RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 470ee95236207b5f929a4f18e230bbfa in your message.)

In [12]:
#read csv
neuro_1 = pd.read_csv("NeuroExp2_1.csv")
neuro_2 = pd.read_csv("NeuroExp2_2.csv")
neuro_3 = pd.read_csv("NeuroExp2_3.csv")

neuro = pd.concat([neuro_1, neuro_2, neuro_3])
len(neuro)
#save csv
neuro.to_csv("NeuroExp2.csv")